In [4]:
import re
import pandas as pd
from konlpy.tag import Komoran
import re
from dateutil import parser
from datetime import datetime, timedelta

In [2]:
data = pd.read_excel('adhd_data.xlsx')
data.head(5)

,sentence,syear,smonth,sday,shour,sminute,fyear,fmonth,fday,fhour,fminute,alarm,todo
0,3월 17일 9시에 헬스장 가기로 했어,2024.0,3.0,17.0,9.0,0.0,2024.0,3.0,17.0,9.0,0.0,NaN,헬스장
1,4월 5일 오후 3시에 친구와 점심 약속이 있어,2024.0,4.0,5.0,15.0,0.0,2024.0,4.0,5.0,15.0,0.0,NaN,친구와 점심 약속
2,5월 12일 오전 10시에 회사 세미나에 참석할 예정이야,2024.0,5.0,12.0,10.0,0.0,2024.0,5.0,12.0,10.0,0.0,NaN,회사 세미나
3,6월 8일 오후 2시에 친구의 생일 파티에 초대받았어,2024.0,6.0,8.0,14.0,0.0,2024.0,6.0,8.0,14.0,0.0,NaN,친구의 생일파티
4,7월 20일 오후 5시에 치과 진료 예약이 있어,2024.0,7.0,20.0,17.0,0.0,2024.0,7.0,20.0,17.0,0.0,NaN,치과 진료 예약


코드 초안

맨 마지막 코드 이용

In [1]:
def extract_schedule_info(sentence):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sentence)  # 특수문자 제거
    text = text.replace(" ", "")  # 모든 공백 제거
    
    # 시작 기간을 추출하는 정규표현식
    start_time_pattern = r'(?:(어제|오늘|내일|모레|\d+년))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후))?(?:(\d+)시)?(?:(\d+)분)?(?:부터)?' # 시작 기간 추출
    match = re.search(start_time_pattern, text)
    if match:
        syear = match.group(1) if match.group(1) else datetime.now().year  # 년도를 추출합니다. 년도가 없으면 기본값으로 2024를 사용합니다.
        smonth = match.group(2).strip("월") if match.group(2) else datetime.now().month
        sday = match.group(3).strip("일") if match.group(3) else datetime.now().day
        
        
        
        
        stime_period = match.group(4)
        shour = match.group(5) if match.group(5) else "0"
        if stime_period == '오후' and int(shour) < 12:
            shour = str(int(shour) + 12)
        if not stime_period:
            shour = str(int(shour))    
        sminute = match.group(6) if match.group(6) else "0"
        return syear, smonth, sday, stime_period, shour, sminute
        
    else:
        return None, None, None, None, None

In [19]:
def extract_schedule_info(sentence):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sentence)  # 특수문자 제거
    text = text.replace(" ", "")  # 모든 공백 제거
    
    # 시작 기간을 추출하는 정규표현식
    start_time_pattern = r'(?:(어제|오늘|내일|모레|이번주|다음주|다다음주|\d+년))?(?:(월요일|화요일|수요일|목요일|금요일|토요일|일요일|월|화|수|목|금|토|일))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후|새벽|밤|낮|저녁|아침))?(?:(\d+)시)?(?:(반|\d+분))?(?:부터)?' # 시작 기간 추출
    match = re.search(start_time_pattern, text)
    if match:
        syear = match.group(1) if match.group(1) else datetime.now().year  # 년도를 추출합니다. 년도가 없으면 기본값으로 2024를 사용합니다.
        smonth = match.group(3).strip("월") if match.group(3) else datetime.now().month
        sday = match.group(4).strip("일") if match.group(4) else datetime.now().day
        sdow = match.group(2) 
        
        if match.group(1) == '오늘':
            syear = datetime.now().year
            smonth = datetime.now().month
            sday = datetime.now().day
        elif match.group(1) == '어제':
            syear = (datetime.now() - timedelta(days=1)).year
            smonth = (datetime.now() - timedelta(days=1)).month
            sday = (datetime.now() - timedelta(days=1)).day
        elif match.group(1) == '내일':
            syear = (datetime.now() + timedelta(days=1)).year
            smonth = (datetime.now() + timedelta(days=1)).month
            sday = (datetime.now() + timedelta(days=1)).day
        elif match.group(1) == '모레':
            syear = (datetime.now() + timedelta(days=2)).year
            smonth = (datetime.now() + timedelta(days=2)).month
            sday = (datetime.now() + timedelta(days=2)).day
        elif match.group(1) == '다음주':
            if match.group(2):  # 요일이 주어진 경우
                today = datetime.now().date()
                weekday_str = match.group(2)
                weekdays = {
                    '월요일': 0,
                    '화요일': 1,
                    '수요일': 2,
                    '목요일': 3,
                    '금요일': 4,
                    '토요일': 5,
                    '일요일': 6
                }
                target_weekday = weekdays[weekday_str]
                current_weekday = today.weekday()
                days_to_target = (target_weekday - current_weekday) % 7
                if days_to_target <= 0:  # 이미 해당 요일이 지난 경우
                    days_to_target += 7
                sday = (datetime.now() + timedelta(days=days_to_target)).day
                syear = (datetime.now() - timedelta(days=days_to_target)).year
            else:  # 요일이 주어지지 않은 경우
                syear = (datetime.now() + timedelta(days=7)).year
                smonth = (datetime.now() + timedelta(days=7)).month
                sday = (datetime.now() + timedelta(days=7)).day


            
        elif match.group(1) == '다다음주':
            syear = (datetime.now() + timedelta(days=14)).year
            smonth = (datetime.now() + timedelta(days=14)).month
            sday = (datetime.now() + timedelta(days=14)).day
        elif match.group(1) == '이번주':
            today = datetime.now().date()
            weekday_str = match.group(2)
            weekdays = {
                '월요일': 0,
                '화요일': 1,
                '수요일': 2,
                '목요일': 3,
                '금요일': 4,
                '토요일': 5,
                '일요일': 6
            }
            target_weekday = weekdays[weekday_str]
            current_weekday = today.weekday()
            days_to_target = (target_weekday - current_weekday) % 7
            sday = (datetime.now() + timedelta(days=days_to_target)).day
            syear = (datetime.now() - timedelta(days=days_to_target)).year
        elif match.group(1) and '년' in match.group(1):
            syear = match.group(1).strip("년")
        else:
            syear = datetime.now().year   
            
            
        stime_period = match.group(5)
        shour = match.group(6) if match.group(6) else "0"
        if (stime_period == '오후' or stime_period == '저녁' or stime_period == '밤' or stime_period == '낮') and int(shour) < 12:
            shour = int(shour) + 12
        if not stime_period:
            shour = int(shour)    
        sminute = match.group(7).strip("분") if match.group(7) else "0"
        if match.group(7) and '반' in match.group(7):
            sminute = 30
        
        # 종료 기간을 추출하는 정규표현식
        end_time_pattern = r'(?:(어제|오늘|내일|모레|다음주|다다음주|\d+년))?(?:(월|화|수|목|금|토|일|월요일|화요일|수요일|목요일|금요일|토요일|일요일))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후|새벽|밤|낮|저녁|아침))?(?:(\d+)시)?(?:(반|\d+분))?((까지|에))' # 종료 기간 추출
        end_match = re.search(end_time_pattern, text)
        if end_match:
            fyear = end_match.group(1) if end_match.group(1) else syear  # 종료 년도를 추출합니다.
            fmonth = end_match.group(3).strip("월") if end_match.group(3) else smonth
            fday = end_match.group(4).strip("일") if end_match.group(4) else sday
            ftime_period = end_match.group(5)
            fhour = end_match.group(6) if end_match.group(6) else "0"
            if (ftime_period == '오후' or ftime_period == '저녁' or ftime_period == '밤' or ftime_period == '낮') and int(fhour) < 12:
                fhour = int(fhour) + 12
            if not ftime_period:
                fhour = int(fhour)    
            fminute = end_match.group(7).strip("분") if end_match.group(7) else "0"
            if end_match.group(7) and '반' in end_match.group(7):
                fminute = 30
            
        else:
            fyear, fmonth, fday, ftime_period, fhour, fminute = syear, smonth, sday, stime_period, shour, sminute

        return syear, smonth, sday, stime_period, shour, sminute, fyear, fmonth, fday, ftime_period, fhour, fminute
    else:
        return None, None, None, None, None, None, None, None, None, None, None, None

In [37]:
def extract_schedule_event(sentence):
    # KoNLPy 형태소 분석기 초기화
    komoran = Komoran()

    # KoNLPy를 사용하여 명사 추출
    nouns = komoran.nouns(sentence)

    # "내일", "오전", "오후", "시" 및 날짜 전용 명사들을 제외한 명사로 이루어진 일정 정보 추출
    excluded_words = r'(?:어제|오늘|내일|모레|이번주|다음주|다다음주|\d+년|\d+월|\d+일|\d+시|\d+분|오후|오전|새벽|월요일|화요일|수요일|목요일|금요일|토요일|일요일|까지|부터|에서|에|합니다|했어|할거야|을)'
    excluded_words = re.sub(r'[ㄱ-ㅎ]', '', excluded_words)
    event_words = [word for word in nouns if word not in excluded_words and not re.match(r'\d+[월일시]', word)]

    # "에" 다음에 오는 명사로 이루어진 일정 정보 추출
    event_match = re.search(r'(?<=에 ).*?(?<![\w\d가-힣])\.$', sentence)
    if event_match:
        event_words.extend([word for word in event_match.group().split() if word not in excluded_words and not re.match(r'\d+[월일시]', word)])

    # 주기적인 일정 처리
    # "매주" 또는 "매월"이 포함되어 있으면 해당 단어 제외
    event_words = [word for word in event_words if word not in ["매주", "매월"]]

    # "다음 달"이라는 표현이 나오면 "달"을 명사에서 제외
    if "다음 달" in sentence:
        event_words.remove("달")

    # "일"이라는 표현이 "마감" 뒤에 오면 "일"을 명사에서 제외
    if "일" in event_words and "마감" in event_words:
        event_words.remove("일")

    # Exclude the word following a number if "일" immediately follows
    for i in range(len(event_words) - 1):
        if re.match(r'\d+', event_words[i]) and event_words[i + 1] == "일":
            event_words.pop(i + 1)

    # Remove leading "일" only if it appears at the beginning
    if event_words and event_words[0] == "일":
        event_words = event_words[1:]

    event = " ".join(event_words) if event_words else "todo"

    return event

In [24]:
'''
import re

def extract_schedule_event(sentence):
    # 입력 문장에서 특수 문자와 공백을 제거합니다.
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sentence)
    text = re.sub(r'[ㄱ-ㅎ]', '', text)
    text = text.replace(" ", "")  
    
    # 키워드를 추출하기 위한 정규표현식 패턴을 정의합니다.
    keyword_pattern = r'(?:어제|오늘|내일|모레|이번주|다음주|다다음주|\d+년|\d+월|\d+일|\d+시|\d+분|오후|오전|새벽|월요일|화요일|수요일|목요일|금요일|토요일|일요일|까지|부터|에서|에|합니다|했어|할거야|을)'
    
    # text에서 keyword_pattern에 해당하는 부분을 삭제합니다.
    extracted_text = re.sub(keyword_pattern, '', text)
    
    return extracted_text
    '''

In [38]:
date = extract_schedule_info("4월 19일에 경복궁 가기로했어")
print(date)

(2024, '4', '19', None, 0, '0', 2024, '4', '19', None, 0, '0')


In [40]:
event = extract_schedule_event("4월 19일에 경복궁 가기로했어")
print(event)

경복궁


In [43]:
date = extract_schedule_info("4월 20일부터 4월 23일 5시 30분까지 졸프 과제 있어")
print(date)

(2024, '4', '20', None, 0, '0', 2024, '4', '23', None, 5, '30')


In [45]:
event = extract_schedule_event("4월 20일부터 4월 23일 5시 30분까지 졸업 프로젝트 과제 있어")
print(event)

졸업 프로젝트 과제


In [25]:
# 각 일정을 추출하고 리스트에 저장합니다.
schedules = []
events = []
for i in range(len(data)):
    schedule_info = extract_schedule_info(data.loc[i,'sentence'])
    schedule_events = extract_schedule_event(data.loc[i,'sentence'])
    if schedule_info:
        schedules.append(schedule_info)
        events.append(schedule_events)
        

# 엑셀 파일에 쓰기 위한 코드
import pandas as pd

# 추출된 일정 정보를 데이터프레임으로 변환
df = pd.DataFrame(schedules, columns=['syear', 'smonth', 'sday', 'stime_period', 'shour', 'sminute', 'fyear', 'fmonth', 'fday', 'ftime_period', 'fhour', 'fminute'])
event = pd.DataFrame(events, columns=['todo'])

In [26]:
pd.set_option('display.max_rows', None)
sen = pd.DataFrame(data['sentence'], columns=['Sentence'])
df = pd.concat([pd.DataFrame(data['sentence']),df,event], axis=1)
df

,sentence,syear,smonth,sday,stime_period,shour,sminute,fyear,fmonth,fday,ftime_period,fhour,fminute,todo
0,3월 17일 9시에 헬스장 가기로 했어,2024,3,17,None,9,0,2024,3,17,None,9,0,헬스장가기로
1,4월 5일 오후 3시에 친구와 점심 약속이 있어,2024,4,5,오후,15,0,2024,4,5,오후,15,0,친구와점심약속이있어
2,5월 12일 오전 10시에 회사 세미나에 참석할 예정이야,2024,5,12,오전,10,0,2024,5,12,오전,10,0,회사세미나참석할예정이야
3,6월 8일 오후 2시에 친구의 생일 파티에 초대받았어,2024,6,8,오후,14,0,2024,6,8,오후,14,0,친구의생일파티초대받았어
4,7월 20일 오후 5시에 치과 진료 예약이 있어,2024,7,20,오후,17,0,2024,7,20,오후,17,0,치과진료예약이있어
5,8월 3일 오전 11시에 도서관에서 책을 빌려야 해,2024,8,3,오전,11,0,2024,8,3,오전,11,0,도서관책빌려야해
6,9월 15일 오후 1시에 부모님과 저녁 식사 예정이에요,2024,9,15,오후,13,0,2024,9,15,오후,13,0,부모님과저녁식사예정이요
7,10월 10일 오전 9시에 새로운 프로젝트 회의가 있을 거예요,2024,10,10,오전,9,0,2024,10,10,오전,9,0,새로운프로젝트회의가있거예요
8,11월 22일 오후 4시에 팀 빌딩 활동 참여할 계획이에요,2024,11,22,오후,16,0,2024,11,22,오후,16,0,팀빌딩활동참여할계획이요
9,12월 7일 오전 10시에 친구와 커피를 마시러 갈 거예요,2024,12,7,오전,10,0,2024,12,7,오전,10,0,친구와커피를마시러갈거예요


In [54]:
datetime.today().weekday()

5